In [ ]:
import os
import warnings
import sys
import codecs
import numpy as np
import argparse
import json
import pickle

from util import read_passages, evaluate, make_folds, clean_words, test_f1, to_BIO, from_BIO, from_BIO_ind, arg2param

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1.0
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
import keras.backend as K
K.set_session(sess)
from keras.activations import softmax
from keras.regularizers import l2
from keras.models import Model, model_from_json
from keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed, Bidirectional
from keras.callbacks import EarlyStopping,LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam, RMSprop, SGD
from crf import CRF
from attention import TensorAttention
from custom_layers import HigherOrderTimeDistributedDense
from generator import BertDiscourseGenerator
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from discourse_tagger_generator_bert import PassageTagger

import matplotlib.pyplot as plt
from scipy.special import softmax
from matplotlib import transforms

In [ ]:
use_attention = True
att_context = "LSTM_clause"
bidirectional = bid = True
crf = True
lstm = False
maxseqlen = 40
maxclauselen = 60
input_size = 768
embedding_dropout=0.4 
high_dense_dropout=0.4
attention_dropout=0.6
lstm_dropout=0.5
word_proj_dim=300 
hard_k=0 
lstm_dim = 350 
rec_hid_dim = 75 
att_proj_dim = 200 
batch_size = 10
reg=0


In [ ]:
prefix="scidt_scibert/"
model_ext = "att=%s_cont=%s_lstm=%s_bi=%s_crf=%s"%(str(use_attention), att_context, str(lstm), str(bid), str(crf))
model_config_file = open(prefix+"model_%s_config.json"%model_ext, "r")
model_weights_file_name = prefix+"model_%s_weights"%model_ext
model_label_ind = prefix+"model_%s_label_ind.json"%model_ext
label_ind_json = json.load(open(model_label_ind))
label_ind = {k: int(label_ind_json[k]) for k in label_ind_json}
num_classes = len(label_ind)

In [ ]:
if use_attention:
    inputs = Input(shape=(maxseqlen, maxclauselen, input_size))
    x = Dropout(embedding_dropout)(inputs)
    x = HigherOrderTimeDistributedDense(input_dim=input_size, output_dim=word_proj_dim, reg=reg)(x)
    att_input_shape = (maxseqlen, maxclauselen, word_proj_dim)
    x = Dropout(high_dense_dropout)(x)
    x, raw_attention = TensorAttention(att_input_shape, context=att_context, hard_k=hard_k, proj_dim = att_proj_dim, rec_hid_dim = rec_hid_dim, return_attention=True)(x)
    x = Dropout(attention_dropout)(x)
else:
    inputs = Input(shape=(maxseqlen, input_size))
    x = Dropout(embedding_dropout)(inputs)
    x = TimeDistributed(Dense(input_dim=input_size, units=word_proj_dim))

if bidirectional:
    x = Bidirectional(LSTM(input_shape=(maxseqlen,word_proj_dim), units=lstm_dim, 
                                  return_sequences=True,kernel_regularizer=l2(reg),
                                  recurrent_regularizer=l2(reg), 
                                  bias_regularizer=l2(reg)))(x)
    x = Dropout(lstm_dropout)(x) 
elif lstm:
    x = LSTM(input_shape=(maxseqlen,word_proj_dim), units=lstm_dim, return_sequences=True,
                    kernel_regularizer=l2(reg),
                    recurrent_regularizer=l2(reg), 
                    bias_regularizer=l2(reg))(x)
    x = Dropout(lstm_dropout)(x) 

if crf:
    Crf = CRF(num_classes,learn_mode="join")
    discourse_prediction = Crf(x)
    tagger = Model(inputs=inputs, outputs=[discourse_prediction])        
else:
    discourse_prediction = TimeDistributed(Dense(num_classes, activation='softmax'),name='discourse')(x)
    tagger = Model(inputs=inputs, outputs=[discourse_prediction])

In [ ]:
tagger.load_weights(model_weights_file_name)


In [ ]:
if crf:
    tagger.compile(optimizer=Adam(), loss=Crf.loss_function, metrics=[Crf.accuracy])
else:
    tagger.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
inp = tagger.input
attention_output = tagger.layers[4].output[1]

In [ ]:
functor = K.function([inp, K.learning_phase()], [attention_output])

In [ ]:
test_file = "lucky_test.txt"

In [ ]:
params = {
    "repfile":"/nas/home/xiangcil/scibert_scivocab_uncased",
    "use_attention": True,
    "batch_size": 10,
    "maxseqlen": 40,
    "maxclauselen": 60
         }

In [ ]:
pretrained_path = params["repfile"]
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

bert = load_trained_model_from_checkpoint(config_path, checkpoint_path)
bert._make_predict_function() # Crucial step, otherwise TF will give error.

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)  

In [ ]:
str_seqs, label_seqs = read_passages(test_file, is_labeled=True)
str_seqs = clean_words(str_seqs)
label_seqs = to_BIO(label_seqs)

In [ ]:
bert_generator = BertDiscourseGenerator(bert, tokenizer, str_seqs, label_seqs, label_ind, 10, True, 40, 60, True, input_size=768)

In [ ]:
test_X, test_Y = bert_generator.make_data(str_seqs, label_seqs)

In [ ]:
attention_raw_scores = functor([test_X])[0]

In [ ]:
attention_scores = softmax(attention_raw_scores,axis=-1)

In [ ]:
reverse_token_dict = {v:k for k,v in token_dict.items()}

In [ ]:
str_seqs_tokenized = []
for str_seq in str_seqs:
    str_seq_tokenized = []
    for clause in str_seq:
        clause_tokenized = []
        indices, segments = tokenizer.encode(clause.lower(), max_len=512)
        for i in range(60):
            clause_tokenized.append(reverse_token_dict[indices[i]])
        str_seq_tokenized.append(clause_tokenized)
    str_seqs_tokenized.append(str_seq_tokenized)

In [ ]:
plt.figure()
plt.imshow(attention_raw_scores[0,:,:],cmap='gray',aspect="equal",interpolation='none')
plt.ylabel("discourse ID")
plt.xlabel("word ID")
plt.colorbar()
plt.show()

In [ ]:
goal_string = str_seqs_tokenized[5][-5][1:10]
goal_weight = attention_raw_scores[5,-5,1:10]

In [ ]:
# goal
for token, weight in zip(str_seqs_tokenized[5][-5], attention_scores[5,-5,:]):
    print(token, weight)

In [ ]:
method_string = str_seqs_tokenized[5][-4][1:12]
method_weight = attention_raw_scores[5,-4,1:12]

In [ ]:
# method
for token, weight in zip(str_seqs_tokenized[5][-4], attention_scores[5,-4,:]):
    print(token, weight)

In [ ]:
results_string = str_seqs_tokenized[5][-3][1:14]
results_weight = attention_raw_scores[5,-3,1:14]

In [ ]:
results_weight

In [ ]:
# results
for token, weight in zip(str_seqs_tokenized[5][-3], attention_scores[5,-3,:]):
    print(token, weight)

In [ ]:
label_seqs = from_BIO(label_seqs)

In [ ]:
for i, (str_seq, label_seq) in enumerate(zip(str_seqs, label_seqs)):
    print(i,np.mean([len(clause.split()) for clause in str_seq]), len(set(label_seq)))

In [ ]:
def normalize(array):
    low = np.min(array)
    high = np.max(array)
    return (array - low) / (high - low)

In [ ]:
def attention_text(x, y, strings, alphas, orientation='horizontal',
                 ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    ax.axis('off')
    t = ax.transData
    canvas = ax.figure.canvas

    assert orientation in ['horizontal', 'vertical']
    if orientation == 'vertical':
        kwargs.update(rotation=90, verticalalignment='bottom')

    for s, alpha in zip(strings, alphas):
        text = ax.text(x, y, s + " ", alpha=alpha, transform=t, **kwargs)

        # Need to draw to update the text position.
        text.draw(canvas.get_renderer())
        ex = text.get_window_extent()
        if orientation == 'horizontal':
            t = transforms.offset_copy(
                text.get_transform(), x=ex.width, units='dots')
        else:
            t = transforms.offset_copy(
                text.get_transform(), y=ex.height, units='dots')


words = "all unicorns poop rainbows ! ! !".split()
alphas = np.arange(1,9) * 0.1
fig = plt.figure(figsize=(6, 0.3))
attention_text(0, 3, goal_string, normalize(goal_weight), size=18)
attention_text(0, 1.5, method_string, normalize(method_weight), size=18)
attention_text(0, 0, results_string, normalize(results_weight), size=18)
#plt.show()
filename = "attention_visualization.pdf"
plt.savefig(filename,quality=100,bbox_inches='tight')